In [ ]:
import pandas as pd
import xarray as xr
import netCDF4 as nc
from pathlib import Path
from datetime import datetime as dt
import geoviews as gv
import numpy as np
import param
from holoviews.operation.datashader import datashade
import panel as pn
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
pn.extension()

In [ ]:
data_dir = Path('../COVID-19/csse_covid_19_data')
time_series_path = data_dir / 'csse_covid_19_time_series' / 'time_series_19-covid-{VAR}.csv'
daily_reports_path = data_dir / 'csse_covid_19_daily_reports' / '{DATE}.csv'
VARS = ['Confirmed', 'Deaths', 'Recovered']

data_vars = list()

dims = ['Quantity', 'Region', 'Date']
for var in VARS:
    df = pd.read_csv(time_series_path.as_posix().format(VAR=var))
    data_vars.append(df.iloc[:,4:].values)
data_vars.append(data_vars[0] - data_vars[1] - data_vars[2])   # active cases
VARS.append('Active')
data_vars.append(np.diff(data_vars[0], n=1, prepend=0))  # new cases
VARS.append('New')

data_vars={'counts': (dims, np.stack(data_vars))}

coords = dict(
    Country=('Region', df['Country/Region']),
    Province_State=('Region', df['Province/State']),
    Date=df.columns.tolist()[4:],
    longitude=('Region', df['Long']),
    latitude=('Region', df['Lat']),
    Quantity=VARS,
)
data = xr.Dataset(data_vars=data_vars, coords=coords)
data

In [ ]:
cdata = data.groupby('Country').sum('Region')
ds = hv.Dataset(cdata, kdims=['Country', 'Date', 'Quantity'], 
                vdims=['counts'])
dates = data['Date'].values.tolist()
tiles = gv.tile_sources.CartoDark

class CovidViewer(param.Parameterized):
    date = param.String(default=data['Date'].values[0])
    countries = param.ListSelector(default=['China', 'Italy', 'US', 'France'], 
                                   objects=cdata['Country'].values, precedence=0.2)
    quantity = param.ObjectSelector(default=VARS[0], objects=VARS, precedence=0.1)
    
    @param.depends('countries', 'quantity')
    def curves(self):
        dsel = ds.select(Country=self.countries, Quantity=self.quantity)
        curves = dsel.to(hv.Curve, 'Date', ).options(tools=['hover'])
        return curves.overlay(['Country', 'Quantity']).options(
            legend_position='top_left', 
            width=800,
            height=400,
            xrotation=60,
        )
    
    @param.depends('countries')
    def bars(self):
        dsel = ds.select(Country=self.countries, Date=data['Date'][-1])

        bars = hv.Bars(dsel, kdims=['Country', 'Quantity'])
        return bars.opts(width=800, height=400, stacked=False, show_legend=False, xrotation=60)
    
    def cuves_panel(self):
        return pn.Row(
            pn.Param(
                self,
                parameters=['quantity', 'countries'],
                widgets={'countries': {'height': 800}}, 
                show_name=False
            ),
            pn.Column(
                self.curves,
                self.bars,
                
            ),
        )

    @staticmethod
    def infection_map(date, variable):
        date = data['Date'].values[date]
        seldata = {
            'longitude': data['longitude'], 
            'latitude': data['latitude'], 
            'Confirmed': data.sel(Quantity='Confirmed', Date=date)['counts'],
            'Deaths': data.sel(Quantity='Deaths', Date=date)['counts'],
            'Recovered': data.sel(Quantity='Recovered', Date=date)['counts'],
            'Active': data.sel(Quantity='Active', Date=date)['counts'],
            'New': data.sel(Quantity='New', Date=date)['counts'],
            'Country': data['Country'],
            'State/Province': data['Province_State'],
        }
        points = gv.Points(seldata, kdims=['longitude', 'latitude'], vdims=VARS+['Country', 'State/Province'])
        return tiles * points.opts(
            size=gv.dim(variable).log()*5,
            logz=True,
            fill_color=variable,
            fill_alpha=.5,
            line_color='gray', 
            cmap='fire_r',
            width=1200, height=600, 
            global_extent=True, 
            tools=['hover'],
            active_tools=['pan', 'wheel_zoom'],
            show_legend=False,
        )
    
    def update_date(self, e):
        self.date = data['Date'].values[e.obj.value]
    
    @param.depends('date')
    def date_label(self):
        return f'## {self.date}'
    
    def summary_panel(self):
        img = 'http://wp.sbcounty.gov/cao/countywire/wp-content/uploads/2020/03/banner.png'
        confirmed_total = int(data.sel(Quantity='Confirmed').isel(Date=-1)['counts'].sum())
        deaths_total = int(data.sel(Quantity='Deaths').isel(Date=-1)['counts'].sum())
        recovered_total = int(data.sel(Quantity='Recovered').isel(Date=-1)['counts'].sum())
        active_total = int(data.sel(Quantity='Active').isel(Date=-1)['counts'].sum())
        new_cases = confirmed_total - int(data.sel(Quantity='Confirmed').isel(Date=-2)['counts'].sum())
        
        
        return pn.Row(
            pn.panel(img),
            pn.pane.HTML(
                f'<h1>Confirmed: {confirmed_total:,}</h1>'
                f'<h1 style="color:gray;">Deaths: {deaths_total:,}</h1>'
                f'<h1 style="color:green;">Recovered: {recovered_total:,}</h1>'
                f'<h1 style="color:red;">Active: {active_total:,}</h1>'
                f'<h1 style="color:blue;">New Cases: {new_cases:,}</h1>'
            )
        )
    
    def map_panel(self):
        map_panel = pn.panel(hv.DynamicMap(self.infection_map, kdims=['day', 'variable']
                     ).redim.range(day=(0,data['Date'].shape[0]-1)).redim.values(variable=VARS))
        
        map_panel[1][0][0].param.watch(self.update_date, 'value')
        
        return pn.Column(map_panel[0], self.date_label, map_panel[1])
    
    def panel(self):
        return pn.Tabs(
            ('Summary', self.summary_panel()),
            ('Map', self.map_panel()),
            ('Country Plots', self.cuves_panel())
            
        )

cv = CovidViewer()
cv.panel().show()